In [1]:
# Dependencies
import gmaps
import requests
import pandas as pd
import datetime
from api_keys import gkey

### Data Retrieval

In [2]:
# Query url from COHGIS Open Data Portal
url = "https://opendata.arcgis.com/datasets/8d515a90e80840b3bc7a3ada352b0d15_0.geojson"

# Get HPD recent crime data
response = requests.get(url)

# Convert "response" to JSON format
crime_report = response.json()

# Preview "crime_report"
crime_report

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'OBJECTID': 1001,
    'Join_Count': 1,
    'TARGET_FID': 1001,
    'Join_Count_1': 1,
    'Incident_No': 33788619,
    'Time_Begun': '2019-03-17T16:06:00.000Z',
    'UCR_No': 6143,
    'HPD_Beat': '15E40',
    'HPD_District': '15',
    'Offense': 'Theft',
    'Address_Range': '2500-2599 S LOOP W',
    'Premise_Type': 'Restaurant or Cafeteria P',
    'X_Coord': 3106127.89573,
    'Y_Coord': 13811273.98095,
    'Zip_Code': '77054',
    'Time_Frame': 2,
    'SNB_No': 35,
    'SNB_Name': 'SOUTH MAIN',
    'Council_District': 'K',
    'HPD_Division': 'Southwest Division'},
   'geometry': {'type': 'Point',
    'coordinates': [-95.41677792670406, 29.677404853365786]}},
  {'type': 'Feature',
   'properties': {'OBJECTID': 1002,
    'Join_Count': 1,
    'TARGET_FID': 1002,
    'Join_Count_1': 1,
    'Incident_No': 23193419,
    'Time_Begun': '2019-02-21T14:45:00.000Z',
    'UCR_No': 6013,
    'HPD_Beat': '16E10',
 

In [3]:
# Grab "features" data from "crime_report"
crime_list = crime_report["features"]

# Check the length of "crime_list"
len(crime_list)

7368

### Data rearrangement in Pandas DataFrame

In [4]:
# Generate a new empty Pandas dataframe with "Offense", "Crime Type", "Lat", and "Lng" columns
crime_info_pd = pd.DataFrame(columns = ["Offense", "Crime Type", "Lat", "Lng"])

# Preview "crime_info_pd"
crime_info_pd

,Offense,Crime Type,Lat,Lng


In [5]:
# list to store types of crimes
crime_type = []

# Loop through "crime_list"
for case in crime_list:
    
    # Append unique values of crime types to "crime_type" list
    if case["properties"]["Offense"] not in crime_type:
        crime_type.append(case["properties"]["Offense"])

# Get the list of crime type documented in HPD recent crime data
crime_type

['Theft',
 'Robbery',
 'Burglary',
 'Auto Theft',
 'Aggravated Assault',
 'Rape',
 'Murder']

In [6]:
# List for violet crimes
violet_crime = ["Aggravated Assault", "Murder", "Robbery", "Rape"]

# List for non-violet crimes
non_violet_crime = ["Theft", "Burglary", "Auto Theft"]

# List to store date of crime
crime_date = []

# Loop through "crime_list"
for case in crime_list:

    # Note that the length of "crime_info_pd" immediately gets changed once a value is added in a column
    # Use "im" to store the length of "crime_info_pd" so that new values will be added in the same row for each column within the same iteration
    im = len(crime_info_pd)
    
    # Append data to "Offense" column
    crime_info_pd.loc[im, "Offense"] = case["properties"]["Offense"]
    
    # Determine crime type (violet/non-violet) through comparing with "violet_crime" and "non_violet_crime" lists
    if case["properties"]["Offense"] in violet_crime:
        crime_info_pd.loc[im, "Crime Type"] = "Violet"
    if case["properties"]["Offense"] in non_violet_crime:
        crime_info_pd.loc[im, "Crime Type"] = "Non-Violet"
    
    # Append longitude value to "Lng" column
    crime_info_pd.loc[im, "Lng"] = case["geometry"]["coordinates"][0]

    # Append Latitude value to "Lat" column
    crime_info_pd.loc[im, "Lat"] = case["geometry"]["coordinates"][1]
    
    # Convert date info. into datetime format and temporatorily assign it to "date_im"
    date_im = datetime.datetime.strptime(case["properties"]["Time_Begun"][:10], "%Y-%m-%d")
    # Append date info into "crime_date" list as "yyyy-mm-dd"
    crime_date.append(date_im.strftime("%Y-%m-%d"))
    
# Preview "crime_info_pd"
crime_info_pd.head()

,Offense,Crime Type,Lat,Lng
0,Theft,Non-Violet,29.6774,-95.4168
1,Theft,Non-Violet,29.614,-95.394
2,Robbery,Violet,29.7982,-95.37
3,Theft,Non-Violet,29.7242,-95.5011
4,Theft,Non-Violet,29.6197,-95.2821


In [7]:
# Remove cases of crime with missing data if any
crime_info_pd = crime_info_pd.dropna(how="any")

# Check the length of "crime_info_pd"
len(crime_info_pd)

7368

In [8]:
# Print the date range of HPD recent crime data 
date_range = str("from " + min(crime_date) + " to " + max(crime_date))

print("Crime Report" + " " + date_range)

Crime Report from 2019-02-21 to 2019-03-20


### Heatmap of Crime Types

In [9]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [10]:
# Store "Lat" and "Lng" into "loc_non_violet" and "loc_violet"
loc_non_violet = crime_info_pd.loc[crime_info_pd["Crime Type"] == "Non-Violet", ["Lat","Lng"]].astype(float)

loc_violet = crime_info_pd.loc[crime_info_pd["Crime Type"] == "Violet", ["Lat","Lng"]].astype(float)

# Check the total number of non-violet crimes as well as violet crimes
print(f"Total number of non-violet crimes is: {len(loc_non_violet)}")
print(f"Total number of violet crimes is: {len(loc_violet)}")

Total number of non-violet crimes is: 6028
Total number of violet crimes is: 1340


In [11]:
# Create heatmap for crime types
fig = gmaps.figure()

# Set parameters for non-violet heatmap
# Note that "weights" is not included as "Non_Violet" in every row is of equal importance
non_violet_layer = gmaps.heatmap_layer(loc_non_violet, point_radius = 30)
# Set the color range of crime heatmap from fuchsia to red
non_violet_layer.gradient = [
    (255, 255, 255, 0),
    (255, 0, 255, 0.6),
    (255, 0, 0, 1.0)
  ]
# Apply non-violet heatmap layer on google map
fig.add_layer(non_violet_layer)

# Set parameters for violet heatmap
# Note that total violet crime is roughly 4.5 folds of that for non-violet crime in quantity, adjust its opacity in order not to mask the heatdot on non-violet heatmap layer
violet_layer = gmaps.heatmap_layer(loc_violet, point_radius = 30, opacity=0.35)
# Set the color range of crime heatmap within the range of yellow (light color)
violet_layer.gradient = [
    (255, 255, 255, 0),
    (128, 128, 0, 0.6),
    (255, 255, 0, 1.0)
  ]
# Apply violet heatmap layer on google map
fig.add_layer(violet_layer)

# Preview "fig"
fig

Figure(layout=FigureLayout(height='420px'))